### The plan:

- Looping through the different tiff files
- Merge them together
- Give 1 to values = 40 (agricultural land) and 0 to other
- Keep only coordinates where you have one :)
- Coordinates are in x,y (EPSG: 3857), I convert to UTM
- use neighbor function to compute closest distance to field

In [19]:

import numpy as np
import xarray as xr
import glob

import geopandas as gpd

import shapely
from shapely.wkt import loads
from shapely import wkt
import pandas as pd

import rasterio
from rasterio.features import shapes
from rasterio.merge import merge
from rasterio.plot import show_hist
from rasterio.plot import show
import rioxarray

from pyproj import Proj, transform


from rasterio.plot import show

In [20]:
# List of all files
list_tif_files = glob.glob("Data_test/Land_cover_masked_26_04/*.tif")
list_tif_test_files = glob.glob("Data_test/*.tif")


In [ ]:
# Merging them all together 
arr, out_trans = merge(list_tif_files)

# remove 1st dimension
arr = arr[0, :, :]

In [12]:
# Take indices where you have agricultural land 
coords = np.where(arr == 1)
# keep indices of coordinates with one
x_ind = list(coords[0])
y_ind = list(coords[1])
# Find the (x,y) coordinates in the raster file
coordinate = rasterio.transform.xy(out_trans,x_ind, x_ind)
x_y_coords = np.dstack((coordinate[0],coordinate[1]))



/usr/lib/python3/dist-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/lib/python3/dist-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
# Remove dimension
x_y_coords = x_y_coords[0, :, :]

# Reproject to UTM
inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')

x2,y2 = transform(inProj,outProj,x_y_coords[:,0],x_y_coords[:,1])

In [15]:
# Make a dataframe with the coordinates
df = pd.DataFrame(x2,y2).reset_index()

In [16]:
df.rename(columns = {"index":"lat", 0:"lon"}, inplace = True)

In [17]:
# Make a geodataframe with the coordinates
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lat, df.lon))

In [18]:
# Take the latest :)))) merged file for the other dateas
df_Survey_merged = pd.read_csv("Data_test/merged_26_04.csv")

In [19]:

def geo_loads(df):
    df['geometry'] = df['geometry'].apply(wkt.loads)
    
def make_geo_frame(df):
    return gpd.GeoDataFrame(df, geometry = 'geometry')

In [20]:
geo_loads(df_Survey_merged)

In [21]:
df_Survey_merged = make_geo_frame(df_Survey_merged)

In [22]:
from sklearn.neighbors import BallTree
import numpy as np

def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)
def nearest_neighbor(left_gdf, right_gdf, return_dist=True):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.
    
    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """
    
    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name
    
    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)
    
    # Parse coordinates from points and insert them into a numpy array as RADIANS
    left_radians = np.array(left_gdf[left_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    
    # Find the nearest points
    # -----------------------
    # closest ==> index in right_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in meters)
    
    closest, dist = get_nearest(src_points=left_radians, candidates=right_radians)

    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    closest_points = right.loc[closest]
    
    
    # Ensure that the index corresponds the one in left_gdf
    closest_points = closest_points.reset_index(drop=True)
    
    # Add distance if requested 
    if return_dist:
        # Convert to meters from radians
        earth_radius = 6371000  # meters
        closest_points['distance'] = dist * earth_radius
        
    return closest_points

In [23]:
# dataframe with closest agricultural land 
df_near = nearest_neighbor(df_Survey_merged, gdf)

In [24]:
df_Survey_merged["dist_agricultural"] = df_near["distance"]

In [25]:
df_Survey_merged.to_csv("Data_test/merged_with_agr_23_04.csv",index = False)